# Reading Kubios Cloud
26.2.2021, Sakari Lukkarinen\
Metropolia UAS

## Introduction

The following code is based on the example code (see Kubioscloud example for authorization).

### How to use this Notebook

- For USERNAME and PASSWORD use the Kubios HRV application username and password.
- You can use also the same username and password to login into the Kubios Cloud API.
- In addition you need CLIENT_ID to read the data from Kubioscloud.
- Run the code step-by-step.
- Some of the following code snippets needs info/data from the previous steps, so read carefully the comments before running the code cells.
- At the end you should have RR- or PP-interval data of one of the selected measurements done with Kubios HRV application and graphical presentations how the heart rate is variating from cycle-to-cycle.

### References

- [Kubioscloud example for authorization](https://bitbucket.org/kubios/workspace/snippets/4X95xd/kubioscloud-example-for-authorization-code)
- [Kubios Cloud API](https://analysis.kubioscloud.com/v1/portal/documentation/apis.html#kubioscloud-api-reference)

## Setup

In [ ]:
#!/usr/bin/env python3
"""Kubioscloud example for Authorization code grant"""
import base64
import logging
import re
import uuid
from pprint import pprint

import requests

import matplotlib.pyplot as plt
import numpy as np
import urllib

In [ ]:
# Use your Kubios HRV App username and password
USERNAME = "..."
PASSWORD = "....."
CLIENT_ID = "......"

LOGIN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/login"
TOKEN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/oauth2/token"
REDIRECT_URI = "https://analysis.kubioscloud.com/v1/portal/login"

USER_AGENT = "TestApp 1.0"  # FIXME: Use unique name for your application

## Opening a session

In [ ]:
# Logging info
logging.basicConfig(format="%(asctime)-15s [%(levelname)s]: %(message)s")

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

csrf = str(uuid.uuid4())

# Login data structure
login_data = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "username": USERNAME,
    "password": PASSWORD,
    "response_type": "code",
    "access_type": "offline",
    "_csrf": csrf,
}

# Start a session
session = requests.session()

# Open a session
log.info("Authenticating to '%r' with client_id: %r", LOGIN_URL, CLIENT_ID)
login_response = session.post(
    LOGIN_URL,
    data=login_data,
    allow_redirects=False,
    headers={"Cookie": f"XSRF-TOKEN={csrf}", "User-Agent": USER_AGENT},
)
assert (
    login_response.status_code == 302
), f"Status: {login_response.status_code}, Authentication failed."
code = login_response.headers["Location"].split("=")[1]
log.info("Got code: %r", code)

In [ ]:
# Exchange tokens
log.info("Exchanging code to tokens")
exch_data = {
    "client_id": CLIENT_ID,
    "code": code,
    "redirect_uri": REDIRECT_URI,
    "grant_type": "authorization_code",
}
exch_response = session.post(
    TOKEN_URL, data=exch_data
)
log.info("Status code %r", exch_response.status_code)
tokens = exch_response.json()

In [ ]:
# What are the possible keys?
tokens.keys()

In [ ]:
# When do these tokens expire?
tokens['expires_in']

## Reading info

### User details

In [ ]:
# Read user details and show them user friendly form
log.info("Query for user details to test obtained credentials")
response = session.get(
    "https://analysis.kubioscloud.com/v1/user/self",
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)
pprint(response.json())

In [ ]:
# From the previous details copy and paste the user_id (might be team_id)
USER_ID = '...'
BASE_URL = "https://analysis.kubioscloud.com"

GET_USER_INFORMATION = BASE_URL + "/v1/user/{:}".format(USER_ID)
GET_USER_AVATAR_IMAGE = BASE_URL + "/v1/user/{:}/avatar".format(USER_ID)
SET_USER_AVATAR_IMAGE = BASE_URL + "/v1/user/{:}/avatar".format(USER_ID)
UPDATE_USER_INFORMATION = BASE_URL + "/v1/user/{:}".format(USER_ID)

INIT_MEASUREMENT_SESSION = BASE_URL + "/v2/measure/{:}/session".format(USER_ID)

## List of measurements

There are more options, see: [Get list of measurements](https://analysis.kubioscloud.com/v1/portal/documentation/apis.html#get-list-of-measurements)

In [ ]:
# Get and print the measurements in user friendly format
log.info("Get list of measurements")
response = session.get(
    "https://analysis.kubioscloud.com/v2/measure/self/session",
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)
pprint(response.json())

In [ ]:
# List and print measure subject in user friendly format
log.info("List measure subject")
response = session.get(
    "https://analysis.kubioscloud.com/v2/measure/self/subject",
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)
pprint(response.json())

## Get measurement details

In [ ]:
# From list of measurements copy and paste the measure_id here
log.info("Get measure details")
measure_id = '...'

# Print the details of the measure
response = session.get(
    "https://analysis.kubioscloud.com/v2/measure/self/session/" + measure_id,
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)
pprint(response.json())

## Get list of measurements for selected user

In [ ]:
log.info("Get list of measurements for a user")

response = session.get(
    "https://analysis.kubioscloud.com/v2/measure/" + USER_ID + "/session?from=2021-02-10",
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)
pprint(response.json())

## Get all results for a selected user

In [ ]:
# List and print ALL MEASUREMENTS for the selected user
log.info("Get results")
response = session.get(
    "https://analysis.kubioscloud.com/v1/result/" + USER_ID,
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)

pprint(response.json())

## Select one measure for details

In [ ]:
# Copy and paste one of the measure_id's here and check the details
log.info("Get measure details")
measure_id = '....'

response = session.get(
    "https://analysis.kubioscloud.com/v2/measure/self/session/" + measure_id,
    headers={"Authorization": tokens["id_token"], "User-Agent": USER_AGENT},
)
pprint(response.json())

## Read the data related to measurement

In [ ]:
# Copy and paste the data_url of the selected measurement here to get RR-data
data_url = '...'
data = urllib.request.urlopen(data_url)
byte = data.read(2)
rr = []
while byte:
    rr.append(int.from_bytes(byte, byteorder = "little"))
    byte = data.read(2)
rr = np.array(rr)

### Show the RR plot

In [ ]:
x = np.cumsum(rr)/1000
plt.plot(x, rr)
plt.xlabel('time (s)')
plt.ylabel('RR (ms)')
plt.grid()
plt.show()

### Show the Heart rate plot

In [ ]:
bpm = 60*1000/rr
plt.plot(x, bpm)
plt.xlabel('time (s)')
plt.ylabel('Heart rate (BPM)')
plt.grid()
plt.show()